In [ ]:
import os, sys, time, json, contextlib, io

from m.fileio import *
from m.ui import *
from m.util import *

import ffmpeg


testfile = 'caches/fdumpster.mp4'
testfile2 = 'caches/tallcat.gif'
testfile3 = 'caches/nestest.nes'
testfile4 = 'caches/GtKSwIy.jpeg'

class MediaMetadata(object):
    """ Instance object contains metadata retrieved from file.

    self.e: Error state. If it is None, then the instance is initialized.
            Otherwise, contains an object that subclasses Exception.
    self.width: Frame width.
    self.height: Frame height.
    self.probedata: Full metadata as returned by ffprobe.
    self.stderr: Output from ffprobe if an error happened when it was called.
    """
    def __init__(self, filepath:str):
        self.e = None
        self.filepath = filepath
        self.width = None
        self.height = None
        self.probedata = None
        self.stderr = None
        try:
            if not os.path.exists(filepath):
                raise IOError(f"{filepath} not found.")
            meta = ffmpeg.probe(filepath)
            for stream in meta['streams']:
                if stream['codec_type'] == 'video':
                    self.width = stream['width']
                    self.height = stream['height']
                    self.probedata = meta
                    return
            else:
                raise(ValueError(f"{filepath} contains no recognizeable video."))
            pass
        except Exception as e:
            self.e = e
            if hasattr(e, "stderr"):
                self.stderr = e.stderr

''' TODO: Once you have figured out how to get metadata file, then figure out
    how to open in a manner that allows "easy" conversion. Most options would
    take too much memory, so you'd spend time converting the file into varying
    sizes as required by all decoders. e.g. x3 and x2 variants.
    For x3, the base horizontal resolution is 96, vertical scaled appropriately.
    For x2, the base horizontal resolution is 144, also scaled appropriately.
    This produces a 288 pixel width display, which allows a 16 px margin on
    both sides of a 320x240 display.

    You'll need to decide what to do if scaling up or down a video makes the
    height go out of bounds (>240). Perhaps have an optional argument default
    to None which does autoscaling, and if it errors out on the scaling
    operation, return an error and tell them that the argumen is no longer
    optional. e.g. (crop top/bottom, add sidebars, distort image to fullscreen)
    Maybe eventually add a way to choose which region to display? Even though
    we aren't a video editor. That's video editor territory.

    Try to make these conversions happen in the background. Definitely have an
    option for a callback to have each thread report on progress.


'''
class MediaFile(object):
    DEFAULT_SCREEN_WIDTH = 288
    MAXIMUN_SCREEN_HEIGHT = 240
    def __init__(self, metadataobj:"MediaMetadata", hspan=96, *args, **kwargs):
        '''TODO: Add extra arguments in case video requires adjustment to
        combat possible "TOO TALL" errors.
        '''
        cls = self.__class__
        screen_width = cls.DEFAULT_SCREEN_WIDTH
        max_screen_height = cls.MAXIMUN_SCREEN_HEIGHT
        self.e = None
        self.tootall = False
        if not isinstance(metadataobj, MediaMetadata):
            self.e = TypeError("Input object is not type MediaMetadata")
            return
        if metadataobj.e:
            self.e = metadataobj.e
            return
        if screen_width % metadataobj.width:
            self.e = ValueError("hspan is not a multiple of frame width.")
            return
        scale_factor = screen_width / metadataobj.width
        vspan = metadataobj.height / scale_factor
        if vspan > max_screen_height:
            self.tootall = True
            self.e = ValueError(f"Video too tall. width scaling: {scale_factor}, requested height: {vspan}, maximum height: {max_screen_height}")
        #TODO: Actual transcoding to raw or APNG
        pass





d = MediaMetadata(testfile2)
f = MediaFile(d, 96)
print(f)
'''
errclass = d.e.__class__
if issubclass(errclass, Exception):
    print(d.e)
    if d.stderr:
        print(d.stderr)
else:
    print(f"file {d.filepath} opened. Width: {d.width}, height: {d.height}")
'''





'\nerrclass = d.e.__class__\nif issubclass(errclass, Exception):\n    print(d.e)\n    if d.stderr:\n        print(d.stderr)\nelse:\n    print(f"file {d.filepath} opened. Width: {d.width}, height: {d.height}")\n'